In [2]:
import numpy as np
import random
import time  # Zaman ölçümü için

# Veri setini yükleme fonksiyonu
def load_data(file_path):
    coordinates = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        num_nodes = int(lines[0].strip())  # İlk satır düğüm sayısını içerir
        for line in lines[1:]:
            x, y = map(float, line.strip().split())
            coordinates.append((x, y))
    return num_nodes, coordinates

# Öklid mesafesi hesaplama
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            dist = np.sqrt((coordinates[i][0] - coordinates[j][0])**2 + 
                           (coordinates[i][1] - coordinates[j][1])**2)
            distance_matrix[i][j] = dist
            distance_matrix[j][i] = dist
    return distance_matrix

# Karınca Kolonisi Optimizasyonu (ACO) sınıfı
class AntColonyOptimization:
    def __init__(self, distance_matrix, num_ants, num_iterations, alpha=1, beta=2, evaporation_rate=0.5, Q=100):
        self.distance_matrix = distance_matrix
        self.num_nodes = distance_matrix.shape[0]
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha  # Feromon önemi
        self.beta = beta    # Mesafe önemi
        self.evaporation_rate = evaporation_rate
        self.Q = Q          # Feromon artışı sabiti
        self.pheromone = np.ones((self.num_nodes, self.num_nodes))  # Başlangıç feromon seviyesi

    def run(self):
        best_path = None
        best_cost = float('inf')
        
        for iteration in range(self.num_iterations):
            all_paths = []
            for ant in range(self.num_ants):
                path = self.construct_solution()
                cost = self.calculate_path_cost(path)
                all_paths.append((path, cost))
                
                if cost < best_cost:
                    best_cost = cost
                    best_path = path
            
            # Feromon güncelleme
            self.update_pheromones(all_paths)
        
        return best_path, best_cost

    def construct_solution(self):
        path = []
        start_node = random.randint(0, self.num_nodes - 1)
        path.append(start_node)
        visited = set([start_node])
        
        while len(path) < self.num_nodes:
            current_node = path[-1]
            next_node = self.select_next_node(current_node, visited)
            path.append(next_node)
            visited.add(next_node)
        
        # Başlangıç noktasına dönüş ekleniyor
        path.append(path[0])
        return path

    def select_next_node(self, current_node, visited):
        probabilities = []
        total = 0
        
        for node in range(self.num_nodes):
            if node not in visited:
                prob = (self.pheromone[current_node][node]**self.alpha) * ((1 / self.distance_matrix[current_node][node])**self.beta)
                probabilities.append((node, prob))
                total += prob
        
        probabilities = [(node, prob / total) for node, prob in probabilities]
        selected_node = self.roulette_wheel_selection(probabilities)
        return selected_node

    def roulette_wheel_selection(self, probabilities):
        rand = random.random()
        cumulative_probability = 0
        for node, prob in probabilities:
            cumulative_probability += prob
            if rand <= cumulative_probability:
                return node

    def calculate_path_cost(self, path):
        cost = 0
        for i in range(len(path) - 1):  # Son düğümden başlangıç noktasına dönüş de dahil
            cost += self.distance_matrix[path[i]][path[i + 1]]
        return cost

    def update_pheromones(self, all_paths):
        self.pheromone *= (1 - self.evaporation_rate)
        
        for path, cost in all_paths:
            for i in range(len(path) - 1):
                from_node = path[i]
                to_node = path[i + 1]
                self.pheromone[from_node][to_node] += self.Q / cost
                self.pheromone[to_node][from_node] += self.Q / cost

# Ana program
if __name__ == "__main__":
    # Başlangıç zamanı
    start_time = time.time()

    # Veri dosyasını yükle
    file_path = "tsp_51.txt"  # Veri dosyasının adı
    num_nodes, coordinates = load_data(file_path)
    distance_matrix = calculate_distance_matrix(coordinates)

    # ACO parametreleri
    num_ants = 20
    num_iterations = 100
    alpha = 1
    beta = 2
    evaporation_rate = 0.5
    Q = 100

    # ACO çalıştır
    aco = AntColonyOptimization(distance_matrix, num_ants, num_iterations, alpha, beta, evaporation_rate, Q)
    best_path, best_cost = aco.run()

    # Bitiş zamanı
    elapsed_time = time.time() - start_time

    # Sonuçları yazdır
    print(f"Çözüm bulundu, süre: {elapsed_time:.2f} saniye")
    print("En iyi maliyet:", best_cost)
    print("En iyi yol:", " -> ".join(map(str, best_path)))

Çözüm bulundu, süre: 3.54 saniye
En iyi maliyet: 468.0419688636607
En iyi yol: 33 -> 0 -> 5 -> 2 -> 28 -> 10 -> 9 -> 45 -> 3 -> 46 -> 8 -> 4 -> 35 -> 13 -> 7 -> 19 -> 40 -> 18 -> 11 -> 42 -> 29 -> 43 -> 21 -> 37 -> 20 -> 25 -> 1 -> 31 -> 22 -> 48 -> 32 -> 17 -> 49 -> 39 -> 50 -> 38 -> 15 -> 44 -> 16 -> 14 -> 30 -> 12 -> 23 -> 34 -> 24 -> 41 -> 27 -> 47 -> 26 -> 6 -> 36 -> 33
